# ScalarField クラス入門

このノートブックでは、`gwexpy` の `ScalarField` クラスの基本的な使い方を学びます。

## ScalarField とは？

`ScalarField` は、時間と3次元空間の4次元データを扱うための特殊なクラスです。物理場の時空間構造を表現し、以下の機能を提供します：

- **軸0（時間軸）**: 時間ドメイン ↔ 周波数ドメインの変換
- **軸1-3（空間軸）**: 実空間 ↔ K空間（波数空間）の変換
- **4D構造の保持**: スライシングしても常に4次元を維持
- **バッチ操作**: `FieldList` と `FieldDict` による複数フィールドの一括処理
- **信号処理**: PSD計算、周波数-空間マッピング、相互相関、コヒーレンスマップ


## セットアップ

必要なライブラリをインポートします。


In [1]:
# SWIGLAL警告とその他の警告をクリーンな出力のために抑制
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
warnings.filterwarnings("ignore", category=UserWarning)

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u

from gwexpy.fields import FieldDict, FieldList, ScalarField

# 再現性のためのシード設定
np.random.seed(42)

AttributeError: module 'gwpy.io.registry' has no attribute 'register_reader'

## 1. ScalarField の初期化とメタデータ

`ScalarField` オブジェクトを作成し、そのメタデータを確認します。


In [2]:
# 4Dデータの作成（10時点 × 8×8×8 の空間グリッド）
nt, nx, ny, nz = 10, 8, 8, 8
data = np.random.randn(nt, nx, ny, nz)

# 軸座標の定義
t = np.arange(nt) * 0.1 * u.s
x = np.arange(nx) * 0.5 * u.m
y = np.arange(ny) * 0.5 * u.m
z = np.arange(nz) * 0.5 * u.m

# ScalarField オブジェクトの作成
field = ScalarField(
    data,
    unit=u.V,
    axis0=t,
    axis1=x,
    axis2=y,
    axis3=z,
    axis_names=["t", "x", "y", "z"],
    axis0_domain="time",
    space_domain="real",
)

print(f"Shape: {field.shape}")
print(f"Unit: {field.unit}")
print(f"Axis names: {field.axis_names}")
print(f"Axis0 domain: {field.axis0_domain}")
print(f"Space domains: {field.space_domains}")

NameError: name 'ScalarField' is not defined

### メタデータの確認

- `axis0_domain`: 軸0のドメイン（"time" または "frequency"）
- `space_domains`: 各空間軸のドメイン（"real" または "k"）
- `axis_names`: 各軸の名前

これらのメタデータは、FFT変換時に自動的に更新されます。


## 2. 4D構造を保持するスライシング

`ScalarField` の重要な特徴は、**スライシングしても常に4次元を維持する**ことです。
整数インデックスを使っても、自動的に長さ1のスライスに変換されます。


In [3]:
# 整数インデックスでスライス（通常のndarrayなら3Dになる）
sliced = field[0, :, :, :]

print(f"Original shape: {field.shape}")
print(f"Sliced shape: {sliced.shape}")  # (1, 8, 8, 8) - 4Dを維持！
print(f"Type: {type(sliced)}")  # ScalarField のまま
print(f"Axis names preserved: {sliced.axis_names}")

NameError: name 'field' is not defined

In [4]:
# 複数の軸で整数インデックスを使用
sliced_multi = field[0, 1, 2, 3]

print(f"Multi-sliced shape: {sliced_multi.shape}")  # (1, 1, 1, 1) - やはり4D
print(f"Value: {sliced_multi.value}")

NameError: name 'field' is not defined

この挙動により、ScalarFieldオブジェクトの一貫性が保たれ、メタデータ（軸名やドメイン情報）が失われることがありません。


## 3. 時間-周波数変換（軸0のFFT）

`fft_time()` と `ifft_time()` メソッドを使って、時間軸を周波数軸に変換できます。
GWpy の `TimeSeries.fft()` と同じ正規化を採用しています。


In [5]:
# 時間ドメインのScalarFieldを作成（正弦波）
t_dense = np.arange(128) * 0.01 * u.s
x_small = np.arange(4) * 1.0 * u.m
signal_freq = 10.0  # Hz

# 10 Hz の正弦波を空間的に均一に配置
data_signal = np.sin(2 * np.pi * signal_freq * t_dense.value)[:, None, None, None]
data_signal = np.tile(data_signal, (1, 4, 4, 4))

field_time = ScalarField(
    data_signal,
    unit=u.V,
    axis0=t_dense,
    axis1=x_small,
    axis2=x_small.copy(),
    axis3=x_small.copy(),
    axis_names=["t", "x", "y", "z"],
    axis0_domain="time",
    space_domain="real",
)

# FFT実行
field_freq = field_time.fft_time()

print(f"Time domain shape: {field_time.shape}")
print(f"Frequency domain shape: {field_freq.shape}")
print(f"Axis0 domain changed: {field_time.axis0_domain} → {field_freq.axis0_domain}")

NameError: name 'ScalarField' is not defined

In [6]:
# 周波数スペクトルをプロット（1点のx,y,zを選択）
# Note: スライスしても4D構造が維持されるため、squeeze()で次元を削減
spectrum = np.abs(field_freq[:, 0, 0, 0].value).squeeze()
freqs = field_freq._axis0_index.value

plt.figure(figsize=(10, 4))
plt.plot(freqs, spectrum, "b-", linewidth=2)
plt.axvline(
    signal_freq, color="r", linestyle="--", label=f"{signal_freq} Hz (Input signal)"
)
plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude [V]")
plt.title("FFT Spectrum (ScalarField)")
plt.xlim(0, 50)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ピーク周波数を確認
peak_idx = np.argmax(spectrum)
peak_freq = freqs[peak_idx]
print(f"Peak frequency: {peak_freq:.2f} Hz (expected: {signal_freq} Hz)")

NameError: name 'field_freq' is not defined

### 逆FFT（周波数 → 時間）

`ifft_time()` で元の時間ドメインに戻すことができます。


In [7]:
# 逆FFT
field_reconstructed = field_freq.ifft_time()

# 元の信号と比較
# Note: 4D構造を維持するため、squeeze()で1次元に
original = field_time[:, 0, 0, 0].value.squeeze()
reconstructed = field_reconstructed[:, 0, 0, 0].value.squeeze()

plt.figure(figsize=(10, 4))
plt.plot(t_dense.value, original, "b-", label="Original", alpha=0.7)
plt.plot(t_dense.value, reconstructed.real, "r--", label="Reconstructed", alpha=0.7)
plt.xlabel("Time [s]")
plt.ylabel("Amplitude [V]")
plt.title("IFFT: Frequency → Time")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 誤差を確認
error = np.abs(original - reconstructed.real)
print(f"Max reconstruction error: {np.max(error):.2e} V")

NameError: name 'field_freq' is not defined

## 4. 実空間-K空間変換（空間軸のFFT）

`fft_space()` と `ifft_space()` を使って、空間軸を波数空間（K空間）に変換できます。
角波数 k = 2π / λ の符号付きFFTを使用します。


In [8]:
# 空間的に周期構造を持つデータを作成
nx, ny, nz = 16, 16, 16
x_grid = np.arange(nx) * 0.5 * u.m
y_grid = np.arange(ny) * 0.5 * u.m
z_grid = np.arange(nz) * 0.5 * u.m

# X方向に波長 4m の正弦波
wavelength = 4.0  # m
k_expected = 2 * np.pi / wavelength  # rad/m

data_spatial = np.sin(2 * np.pi * x_grid.value / wavelength)[None, :, None, None]
data_spatial = np.tile(data_spatial, (4, 1, ny, nz))

field_real = ScalarField(
    data_spatial,
    unit=u.V,
    axis0=np.arange(4) * 0.1 * u.s,
    axis1=x_grid,
    axis2=y_grid,
    axis3=z_grid,
    axis_names=["t", "x", "y", "z"],
    axis0_domain="time",
    space_domain="real",
)

# X軸のみFFT
field_kx = field_real.fft_space(axes=["x"])

print(f"Original space domains: {field_real.space_domains}")
print(f"After fft_space: {field_kx.space_domains}")
print(f"Axis names: {field_kx.axis_names}")

NameError: name 'ScalarField' is not defined

In [9]:
# K空間スペクトルをプロット
# Note: squeeze()で次元削減
kx_spectrum = np.abs(field_kx[0, :, 0, 0].value).squeeze()
kx_values = field_kx._axis1_index.value

plt.figure(figsize=(10, 4))
plt.plot(kx_values, kx_spectrum, "b-", linewidth=2)
plt.axvline(k_expected, color="r", linestyle="--", label=f"k = {k_expected:.2f} rad/m")
plt.axvline(-k_expected, color="r", linestyle="--")
plt.xlabel("Wavenumber kx [rad/m]")
plt.ylabel("Amplitude")
plt.title("Spatial FFT: Real → K space")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ピーク波数を確認
peak_idx = np.argmax(kx_spectrum)
peak_k = kx_values[peak_idx]
print(f"Peak wavenumber: {peak_k:.2f} rad/m (expected: ±{k_expected:.2f} rad/m)")

NameError: name 'field_kx' is not defined

### 波長の計算

K空間では、`wavelength()` メソッドで波長を計算できます。


In [10]:
# 波長を計算
wavelengths = field_kx.wavelength("kx")

print(f"Wavelength at k={k_expected:.2f}: {2 * np.pi / k_expected:.2f} m")
print(
    f"Calculated wavelengths range: {wavelengths.value[wavelengths.value > 0].min():.2f} - {wavelengths.value[wavelengths.value < np.inf].max():.2f} m"
)

NameError: name 'field_kx' is not defined

### 全空間軸のFFT

`axes` パラメータを省略すると、全空間軸をまとめてFFTできます。


In [11]:
# 全空間軸をFFT
field_k_all = field_real.fft_space()
print(f"All spatial axes in K space: {field_k_all.space_domains}")

# 逆FFTで元に戻す
field_real_back = field_k_all.ifft_space()
print(f"Back to real space: {field_real_back.space_domains}")

# 再構成誤差
reconstruction_error = np.max(np.abs(field_real.value - field_real_back.value))
print(f"Max reconstruction error: {reconstruction_error:.2e}")

NameError: name 'field_real' is not defined

## 5. FieldList と FieldDict によるバッチ操作

複数の `ScalarField` オブジェクトをまとめて処理するには、`FieldList` または `FieldDict` を使用します。


### FieldList

リスト形式で複数のフィールドを管理し、一括でFFT操作を適用できます。


In [12]:
# 3つの異なる振幅を持つScalarFieldを作成
amplitudes = [1.0, 2.0, 3.0]
fields = []

for amp in amplitudes:
    data_temp = amp * np.random.randn(8, 4, 4, 4)
    field_temp = ScalarField(
        data_temp,
        unit=u.V,
        axis0=np.arange(8) * 0.1 * u.s,
        axis1=np.arange(4) * 0.5 * u.m,
        axis2=np.arange(4) * 0.5 * u.m,
        axis3=np.arange(4) * 0.5 * u.m,
        axis_names=["t", "x", "y", "z"],
        axis0_domain="time",
        space_domain="real",
    )
    fields.append(field_temp)

# FieldList を作成
field_list = FieldList(fields, validate=True)
print(f"Number of fields: {len(field_list)}")

NameError: name 'ScalarField' is not defined

In [13]:
# 一括で時間FFTを実行
field_list_freq = field_list.fft_time_all()

print("All fields transformed to frequency domain:")
for i, field in enumerate(field_list_freq):
    print(f"  Field {i}: axis0_domain = {field.axis0_domain}")

NameError: name 'field_list' is not defined

### FieldDict

辞書形式で名前付きフィールドを管理します。


In [14]:
# 名前付きフィールドの辞書を作成
field_dict = FieldDict(
    {"channel_A": fields[0], "channel_B": fields[1], "channel_C": fields[2]},
    validate=True,
)

print(f"Field names: {list(field_dict.keys())}")

NameError: name 'FieldDict' is not defined

In [15]:
# 一括で空間FFTを実行
field_dict_k = field_dict.fft_space_all(axes=["x", "y"])

for name, field in field_dict_k.items():
    print(f"{name}: {field.space_domains}")

NameError: name 'field_dict' is not defined

## 6. 信号処理と解析

物理場の解析において重要な信号処理手法として、以下を実演します：
1. **PSD計算**: 時間・空間方向のパワースペクトル密度
2. **周波数-空間マッピング**: 空間的に周波数がどう変化するか可視化
3. **相互相関**: 信号の伝搬遅延の推定
4. **コヒーレンスマップ**: 空間的なコヒーレンスの可視化


In [16]:
# シミュレーションデータの生成
# 信号: 30 Hz の正弦波が x=1m から伝搬
# ノイズ: ガウスノイズ
fs = 256 * u.Hz
duration = 4 * u.s
t = np.arange(int((fs * duration).to_value(u.dimensionless_unscaled))) / fs.value
x = np.linspace(0, 5, 20)  # 5m, 20点
y = np.linspace(0, 2, 5)  # 2m, 5点
z = np.array([0])  # z=0

shape = (len(t), len(x), len(y), len(z))
data_sig = np.random.normal(0, 0.1, size=shape)

source_freq = 30.0
source_signal = np.sin(2 * np.pi * source_freq * t)
v_prop = 10.0  # 10 m/s
source_pos = 1.0

for i, xi in enumerate(x):
    dist = abs(xi - source_pos)
    delay = dist / v_prop
    shift = int(delay * fs.value)
    sig_delayed = np.roll(source_signal, shift)
    amp = 1.0 / (1.0 + dist)
    data_sig[:, i, :, 0] += amp * sig_delayed[:, np.newaxis]

field_sig = ScalarField(
    data_sig,
    unit=u.V,
    axis0=t * u.s,
    axis1=x * u.m,
    axis2=y * u.m,
    axis3=z * u.m,
    name="Environmental Field",
)
print(field_sig)

NameError: name 'ScalarField' is not defined

### パワースペクトル密度 (PSD)

特定の位置でのPSDを計算し、周波数成分を確認します。


In [17]:
# ソース位置(x=1m)と遠方(x=4m)でのPSDを比較
psd_source = field_sig.psd(point_or_region=(1.0 * u.m, 0.0 * u.m, 0.0 * u.m))
psd_far = field_sig.psd(point_or_region=(4.0 * u.m, 0.0 * u.m, 0.0 * u.m))

plt.figure(figsize=(10, 6))
psd_source.plot(label="Source (x=1m)")
psd_far.plot(label="Far (x=4m)")
plt.xscale("log")
plt.yscale("log")
plt.title("PSD at Different Locations")
plt.legend()
plt.show()

NameError: name 'field_sig' is not defined

### 周波数-空間マッピング

`plot_freq_space` を使用して、空間軸に沿った周波数スペクトルの変化を可視化します。


In [18]:
field_sig.plot_freq_space(
    space_axis="x",
    fixed_coords={"y": 0.0 * u.m, "z": 0.0 * u.m},
    freq_range=(10, 100),
    log_scale=True,
)

NameError: name 'field_sig' is not defined

### 相互相関と遅延推定

基準点（ソース位置）と他の位置との相互相関を計算し、信号の伝搬遅延を可視化します。


In [19]:
# x=1.0m (ソース) を基準点とする
ref_point = (1.0 * u.m, 0.0 * u.m, 0.0 * u.m)

# 相互相関の計算とプロット
lags, corrs = field_sig.plot_cross_correlation(
    ref_point=ref_point,
    scan_axis="x",
    fixed_coords={"y": 0.0 * u.m, "z": 0.0 * u.m},
    max_lag=0.5,
)

NameError: name 'field_sig' is not defined

傾きがV字型になっていることから、信号が x=1.0m から両側に伝搬していることがわかります。


### コヒーレンスマップ

特定周波数（ここでは30Hz）における空間的なコヒーレンスを表示します。


In [20]:
field_sig.plot_coherence_map(
    target_freq=30.0,
    ref_point=ref_point,
    scan_axis="x",
    fixed_coords={"y": 0.0 * u.m, "z": 0.0 * u.m},
)

NameError: name 'field_sig' is not defined

## 6. 数値的不変量の検証

FFT変換の正確性を検証するため、往復変換（round-trip）が元のデータを再構成できることを確認します。


In [21]:
# 時間FFTの不変量チェック: ifft_time(fft_time(f)) ≈ f
np.random.seed(42)
data_test = np.random.randn(64, 4, 4, 4)
field_test = ScalarField(
    data_test,
    unit=u.V,
    axis0=np.arange(64) * 0.01 * u.s,
    axis1=np.arange(4) * u.m,
    axis2=np.arange(4) * u.m,
    axis3=np.arange(4) * u.m,
    axis_names=["t", "x", "y", "z"],
    axis0_domain="time",
    space_domain="real",
)

# Round-trip
field_roundtrip = field_test.fft_time().ifft_time()

# Check
max_error_time = np.max(np.abs(field_test.value - field_roundtrip.value.real))
print(f"時間FFT Round-trip max error: {max_error_time:.2e}")
assert max_error_time < 1e-10, "Time FFT round-trip failed!"
print("✅ 時間FFT不変量チェック: PASSED")

NameError: name 'ScalarField' is not defined